In [108]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.feature_selection import RFE
import operator
import seaborn as sn
from scipy import stats
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SVMSMOTE
from sklearn.model_selection import cross_validate
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import ADASYN
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB
import pandasql as ps
from sklearn.model_selection import RepeatedStratifiedKFold
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [109]:
df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/TRAIN_CLEARED2.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11028 entries, 0 to 11027
Data columns (total 48 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   NewName                                      11028 non-null  object 
 1   ClubInvolved                                 11028 non-null  object 
 2   League                                       11028 non-null  object 
 3   MainPosition                                 11028 non-null  object 
 4   LeagueInvolved                               11028 non-null  object 
 5   CorrMarketValue                              11028 non-null  float64
 6   Country                                      11028 non-null  object 
 7   Club                                         11028 non-null  object 
 8   Position                                     11028 non-null  object 
 9   Foot                                         11028 non-null  object 
 10

In [110]:
df = df.drop(['NewName', 'ClubInvolved', 'Club', 'ClubAge', 'ClubNumPlayersUsedPerMatch',\
     'ClubGoalPerShotOnTargetPerMatch', 'ClubShotOnTargetAgainstUsedPerMatch', 'ClubPointsPerMatch', 'Fee'], axis = 1)

In [4]:
df.columns

Index(['League', 'MainPosition', 'LeagueInvolved', 'CorrMarketValue',
       'Country', 'Position', 'Foot', 'IntReputation', 'Heading_accuracy',
       'Reactions', 'Balance', 'Jumping', 'Season', 'Years', 'DEGREE', 'Eigv',
       'SP_LA_LIGA', 'EN_PL', 'IT_SERIE_A', 'FR_LIGUE_1', 'GE_BUNDESLIGA',
       'ClubInvolvedAge', 'ClubInvolvedCleanSheetsRate', 'MinutesPlayedPer90',
       'ContractLeft', 'Pace_main', 'Shooting_main', 'Defending_main',
       'Passing_main', 'Physical_main', 'Age', 'ClubInvolvedPointsPerMatch',
       'ClubInvolvedNumPlayersUsedPerMatch',
       'ClubInvolvedGoalPerShotOnTargetPerMatch',
       'ClubInvolvedShotOnTargetAgainstUsedPerMatch', 'GoalsPerMinutesPlayed',
       'AssistsPerMinutesPlayed', 'YellowCardsPerMinutesPlayed',
       'RedCardsPerMinutesPlayed'],
      dtype='object')

In [111]:
df['target'] = 0
df.loc[df.League == 'Italian Serie A', 'target'] = 1

In [112]:
df = df.drop('League', axis=1)

In [113]:
df['target'].value_counts()

0    9335
1    1693
Name: target, dtype: int64

In [114]:
#set feature list and target
feature_list = list(df.columns.values)
feature_list.remove('target')
target = 'target'

# define dataset
X = df[feature_list]
y = df[target]

In [8]:
import category_encoders as ce
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

In [9]:
#Impute missing values
imputer = IterativeImputer()
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

In [10]:
#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

In [11]:
X_imp_st['DEGREE_IT_SERIE_A_MUL'] = X_imp_st['DEGREE'] * X_imp_st['IT_SERIE_A']

In [12]:

# define the model
model = XGBClassifier(n_estimators=50, random_state=1)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

[16:32:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[16:32:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:32:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[16:33:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

[16:33:03] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:33:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:33:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory c

fit_time 13.956954073905944
score_time 0.0657606840133667
test_acc 0.9067829776241337
test_recall 0.5475652627915071
test_auc 0.91111520281724
test_precison 0.7805030955177383
test_f1 0.6429263422317867


In [13]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 13.956954073905944
score_time 0.0657606840133667
test_acc 0.9067829776241337
test_recall 0.5475652627915071
test_auc 0.91111520281724
test_precison 0.7805030955177383
test_f1 0.6429263422317867


DecisionTreeRegressor hyperparameter tuning

In [20]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['DEGREE_IT_SERIE_A_MUL'] = X_imp_st['DEGREE'] * X_imp_st['IT_SERIE_A']

# define the model
model = XGBClassifier(n_estimators=50, random_state=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 22.455101346969606
score_time 0.13125871419906615
test_acc 0.9072369449430937
test_recall 0.5511242603550295
test_auc 0.9105762800996289
test_precison 0.7811913810100524
test_f1 0.6454318297872038


In [21]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 22.455101346969606
score_time 0.13125871419906615
test_acc 0.9072369449430937
test_recall 0.5511242603550295
test_auc 0.9105762800996289
test_precison 0.7811913810100524
test_f1 0.6454318297872038


XGBoost hyperparameter tuning

In [28]:
# define the model with default hyperparameters
model = XGBClassifier()
# define the grid of values to search
grid = dict()
grid['verbosity'] = [0]
grid['random_state'] = [1]
grid['n_estimators'] = [500] #100, 300
grid['learning_rate'] = [0.3] #0.01, 0.1, 0.2 , 0.3
grid['subsample'] = [1] #0.5, 0.75
#grid['max_features'] = [10, 20, 30]
grid['max_depth'] = [6] #3,9
#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring=scoring, refit='f1')
# execute the grid search
grid_result = grid_search.fit(X_imp_st, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

Best: 0.665397 using {'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 500, 'random_state': 1, 'subsample': 1, 'verbosity': 0}


In [29]:
#create df for metrics
columns= ['params', 'mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']
indexes = [x for x in range(len(grid_result.cv_results_['params']))]
metric_df = pd.DataFrame(index = indexes, columns=columns)
metric_df = metric_df.fillna(0)
metric_df['params'] = metric_df['params'].astype(str)
metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']] = metric_df[['mean_test_acc', 'mean_test_auc', 'mean_test_recall', 'mean_test_precison', 'mean_test_f1']].astype(float)

metric_df['params'] = grid_result.cv_results_['params']
metric_df['mean_test_acc'] = grid_result.cv_results_['mean_test_acc']
metric_df['mean_test_recall'] = grid_result.cv_results_['mean_test_recall']
metric_df['mean_test_auc'] = grid_result.cv_results_['mean_test_auc']
metric_df['mean_test_precison'] = grid_result.cv_results_['mean_test_precison']
metric_df['mean_test_f1'] = grid_result.cv_results_['mean_test_f1']

metric_df.sort_values(by='mean_test_f1', ascending=False)

,params,mean_test_acc,mean_test_auc,mean_test_recall,mean_test_precison,mean_test_f1
0,"{'learning_rate': 0.3, 'max_depth': 6, 'n_estimators': 500, 'random_state': 1, 'subsample': 1, 'verbosity': 0}",0.911771,0.914548,0.572978,0.795477,0.665397


In [24]:
pd.set_option('display.max_colwidth', None)

In [30]:
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)
m = model.fit(X_imp_st , y)
zip_iterator = zip(list(X_imp_st.columns), m.feature_importances_)
feature_dict = dict(zip_iterator)
print(dict( sorted(feature_dict.items(), key=operator.itemgetter(1),reverse=True)))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


{'IT_SERIE_A': 0.4076947, 'FR_LIGUE_1': 0.030908884, 'Country': 0.030017821, 'SP_LA_LIGA': 0.027424531, 'GE_BUNDESLIGA': 0.026031509, 'LeagueInvolved': 0.025995642, 'CorrMarketValue': 0.025063442, 'Foot': 0.021864893, 'DEGREE_IT_SERIE_A_MUL': 0.02144011, 'Age': 0.018472297, 'DEGREE': 0.017179074, 'MinutesPlayedPer90': 0.01613264, 'Defending_main': 0.015771186, 'ContractLeft': 0.014898372, 'Eigv': 0.014569463, 'Years': 0.013978611, 'Passing_main': 0.013970049, 'ClubInvolvedPointsPerMatch': 0.013743727, 'ClubInvolvedAge': 0.013587736, 'EN_PL': 0.013294191, 'Season': 0.013273719, 'RedCardsPerMinutesPlayed': 0.0130339395, 'ClubInvolvedNumPlayersUsedPerMatch': 0.013012548, 'Balance': 0.012884533, 'ClubInvolvedGoalPerShotOnTargetPerMatch': 0.012705534, 'ClubInvolvedCleanSheetsRate': 0.012055651, 'Reactions': 0.011992196, 'AssistsPerMinutesPlayed': 0.011672656, 'Physical_main': 0.011127829, 'MainPosition': 0.011040082, 'Position': 0.011000071, 'Heading_accuracy': 0.010874859, 'IntReputation':

Stacking

In [116]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB

In [135]:
#encoding the categorival variables using target encoding
cbe_encoder = ce.cat_boost.CatBoostEncoder()
X_cbe = cbe_encoder.fit_transform(X,y)

#Impute missing values
imputer = IterativeImputer(estimator = DecisionTreeRegressor(max_depth= None, max_features="sqrt", random_state=1, \
    min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.2, min_impurity_decrease=0.))
imputer.fit(X_cbe)
X_imp = imputer.transform(X_cbe)
X_imp = pd.DataFrame(data = X_imp, columns = X_cbe.columns)

#standardization 
scaler = StandardScaler()
X_imp_st = scaler.fit_transform(X_imp)
X_imp_st = pd.DataFrame(data = X_imp_st, columns=X_imp.columns)

X_imp_st['DEGREE_IT_SERIE_A_MUL'] = X_imp_st['DEGREE'] * X_imp_st['IT_SERIE_A']

In [167]:
from sklearn.ensemble import VotingClassifier

#scores = [0.457720, 0.458433, 0.412003, 0.476421, 0.50272, 0.4567053]
scores = [0.888329, 0.928136, 0.918207, 0.927275, 0.929905, 0.88130]

bernoulli = BernoulliNB(alpha=0.5, binarize=1)
bagging = BaggingClassifier(n_estimators=100)
adab = AdaBoostClassifier(learning_rate=0.7, n_estimators=100)
grad = GradientBoostingClassifier(learning_rate=0.5, max_depth=13, n_estimators=100, random_state=1)
XGB = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)
gaussian =GaussianNB()

estimators_list = [("bernoulli", bernoulli), ("bagging", bagging),\
    ("adab", adab), ("grad", grad), ("XGB", XGB), ("gaussian", gaussian)]

# create the ensemble
model = VotingClassifier(estimators=estimators_list, voting='soft', weights=scores)


In [119]:
# # # define the model
# model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=50, random_state=1,\
#     subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_imp_st , y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 202.3747815489769
score_time 0.6098430156707764
test_acc 0.9083704646459992
test_recall 0.573271841280891
test_auc 0.8886250157386654
test_precison 0.7716364318986947
test_f1 0.6573157253979357


In [121]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 202.3747815489769
score_time 0.6098430156707764
test_acc 0.9083704646459992
test_recall 0.573271841280891
test_auc 0.8886250157386654
test_precison 0.7716364318986947
test_f1 0.6573157253979357


Syntetic data generalization

In [158]:
X_train = X_imp_st.copy()
X_train['target'] = y

positives = X_train[X_train.target == 1]

In [159]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(positives)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:5003: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2494: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats

In [160]:
sample = model.sample(2000)

In [161]:
from sdv.evaluation import evaluate
evaluate(sample, positives, metrics=['CSTest', 'KSTest'], aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,NaN,NaN,0.0,1.0,MAXIMIZE,"Cannot find fields of types ('boolean', 'categorical')"
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.836322,0.836322,0.0,1.0,MAXIMIZE,None


In [162]:
X_synt = X_train.append(sample)

In [163]:
X_synt['target'].value_counts()

0    9335
1    3693
Name: target, dtype: int64

In [164]:
y_synt = X_synt['target']
X_synt = X_synt.drop('target', axis = 1)

In [165]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_synt , y_synt, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)

# report performance
for key, value in n_scores.items():
    print(key, mean(value))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

fit_time 103.04239860773086
score_time 0.14509761333465576
test_acc 0.9241641644650829
test_recall 0.8122218560023438
test_auc 0.9596634270164532
test_precison 0.9107503283589502
test_f1 0.8585412038777221


In [166]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 103.04239860773086
score_time 0.14509761333465576
test_acc 0.9241641644650829
test_recall 0.8122218560023438
test_auc 0.9596634270164532
test_precison 0.9107503283589502
test_f1 0.8585412038777221


SMOTE

In [178]:
from imblearn.over_sampling import SVMSMOTE
#overfit the target label
over = SVMSMOTE(sampling_strategy=0.25)
X_res, y_res = over.fit_resample(X_imp_st, y)

In [52]:
# define the model
model = XGBClassifier(learning_rate=0.3, max_dept=6, max_features=10, n_estimators=500, random_state=1,\
    subsample=1, verbosity=0)

# define the evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=1)

#scoring params
scoring = {'acc': 'accuracy',
           'recall': 'recall',
           'auc': 'roc_auc',
           'precison': 'precision',
           'f1': 'f1'}

# evaluate the model on the dataset
n_scores = cross_validate(model, X_res, y_res, scoring=scoring, cv=cv, n_jobs=-1, return_train_score= False)



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

In [53]:
# report performance
for key, value in n_scores.items():
    print(key, mean(value))

fit_time 77.5805233836174
score_time 0.08300092220306396
test_acc 0.9442272060085017
test_recall 0.8149800796812748
test_auc 0.9692155756393932
test_precison 0.8970385427932932
test_f1 0.8537391876316913


Predict

In [179]:
predict_df = pd.read_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/predict_df_temp3.csv")
predict_df = predict_df.drop('Unnamed: 0', axis = 1)

predict_df['Pace_main'] = round(predict_df['Acceleration'] * 0.45 + predict_df['Sprint_speed'] * 0.55, 0)
predict_df['Shooting_main'] = round(predict_df['Finishing'] * 0.45 + predict_df['Long_shots'] * 0.20 + predict_df['Penalties'] * 0.05 + \
     predict_df['Positioning'] * 0.05 + predict_df['Shot_power'] * 0.20 + predict_df['Volleys'] * 0.05, 0)
predict_df['Dribbling_main'] = round(predict_df['Agility'] * 0.1 + predict_df['Balance'] * 0.05 + predict_df['Ball_control'] * 0.35 + \
     predict_df['Dribbling'] * 0.5, 0)
predict_df['Defending_main'] = round(predict_df['Heading_accuracy'] * 0.1 + predict_df['Interceptions'] * 0.2 + predict_df['Marking'] * 0.3 + \
     predict_df['Sliding_tackle'] * 0.3 + predict_df['Standing_tackle'] * 0.3, 0)
predict_df['Passing_main'] = round(predict_df['Crossing'] * 0.2 + predict_df['Curve'] * 0.05 + predict_df['FK_accuracy'] * 0.05 + \
     predict_df['Long_passing'] * 0.15 + predict_df['Short_passing'] * 0.35 + predict_df['Vision'] * 0.25, 0)
predict_df['Physical_main'] = round(predict_df['Aggression'] * 0.2 + predict_df['Jumping'] * 0.05 + predict_df['Stamina'] * 0.25 + \
     predict_df['Strength'] * 0.5, 0)

gk_predict = predict_df[predict_df.Position == 'GK']
outfielder_predict = predict_df[predict_df.Position != 'GK']

outfielder_predict = outfielder_predict.drop(['GK_diving', 'GK_handling', 'GK_kicking', 'GK_positioning', 'GK_reflexes',\
    'ClubInvolvedSaves', 'ClubInvolvedSaveRate', 'GKMatchPlayed', 'GKStarts', 'GKMin', 'GK90s', 'GKGoalAgainst', 'GKGoalAgainstPer90',\
        'GKSaves', 'GKSavePercent', 'GKWin', 'GKDraw', 'GKLost', 'GKCleanSheet', 'GKCleanSheetPercent'], axis = 1)

outfielder_predict = outfielder_predict.drop(['ShotsOnTargetPercent', 'ClubInvolvedShotsOnTargetRate', 'ClubInvolvedGoalPerShot',\
    'ShotsPer90', 'Shots', 'ClubInvolvedShotsPer90', 'ClubInvolvedShots'], axis = 1)

outfielder_predict = outfielder_predict.drop(['GoalsPerShot', 'ClubInovlvedPossession'], axis = 1)

outfielder_predict['ClubInvolvedPlayed'] = outfielder_predict['ClubInvolvedWin'] + outfielder_predict['ClubInvolvedDraw'] + outfielder_predict['ClubInvolvedLoose']
# team_df['ClubInvolvedPlayed'] = team_df['ClubInvolvedWin'] + team_df['ClubInvolvedDraw'] + team_df['ClubInvolvedLoose']
# outfielder_predict['ClubAge'] = team_df['ClubInvolvedAge']
outfielder_predict['ClubInvolvedWinPerMatch'] = outfielder_predict['ClubInvolvedWin'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedDrawPerMatch'] = outfielder_predict['ClubInvolvedDraw'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedLoosePerMatch'] = outfielder_predict['ClubInvolvedLoose'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalForPerMatch'] = outfielder_predict['ClubInvolvedGoalFor'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalAgainstPerMatch'] = outfielder_predict['ClubInvolvedGoalAgainst'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedPointsPerMatch'] = outfielder_predict['ClubInvolvedPoints'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubPointsPerMatch'] = team_df['ClubInvolvedPoints'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedAssistsPerMatch'] = outfielder_predict['ClubInvolvedAssists'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedNonPenaltyGoalPerMatch'] = outfielder_predict['ClubInvolvedNonPenaltyGoal'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedYellowCardPerMatch'] = outfielder_predict['ClubInvolvedYellowCard'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedRedCardPerMatch'] = outfielder_predict['ClubInvolvedRedCard'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedNumPlayersUsedPerMatch'] = outfielder_predict['ClubInvolvedNumPlayersUsed'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubNumPlayersUsedPerMatch'] = team_df['ClubInvolvedNumPlayersUsed'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedShotsOnTargetPerMatch'] = outfielder_predict['ClubInvolvedShotsOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedGoalPerShotOnTargetPerMatch'] = outfielder_predict['ClubInvolvedGoalPerShotOnTarget'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubGoalPerShotOnTargetPerMatch'] = team_df['ClubInvolvedGoalPerShotOnTarget'] / team_df['ClubInvolvedPlayed']

outfielder_predict['ClubInvolvedShotOnTargetAgainstUsedPerMatch'] = outfielder_predict['ClubInvolvedShotOnTargetAgainst'] / outfielder_predict['ClubInvolvedPlayed']
# outfielder_predict['ClubShotOnTargetAgainstUsedPerMatch'] = team_df['ClubInvolvedShotOnTargetAgainst'] / team_df['ClubInvolvedPlayed']
outfielder_predict['ClubInvolvedCleanSheetsPerMatch'] = outfielder_predict['ClubInvolvedCleanSheets'] / outfielder_predict['ClubInvolvedPlayed']

outfielder_predict['MatchesPlayedPerClubMatches'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['StartsPerClubMatches'] = outfielder_predict['Starts'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutes'] = outfielder_predict['MatchesPlayed'] / outfielder_predict['ClubInvolvedPlayed']
outfielder_predict['GoalsPerMinutesPlayed'] = outfielder_predict['Goals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['AssistsPerMinutesPlayed'] = outfielder_predict['Assists'] / outfielder_predict['MinutesPlayed']
outfielder_predict['NonPenaltyGoalsPerMinutesPlayed'] = outfielder_predict['NonPenaltyGoals'] / outfielder_predict['MinutesPlayed']
outfielder_predict['YellowCardsPerMinutesPlayed'] = outfielder_predict['YellowCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['RedCardsPerMinutesPlayed'] = outfielder_predict['RedCards'] / outfielder_predict['MinutesPlayed']
outfielder_predict['ShotsOnTargetPerMinutesPlayed'] = outfielder_predict['ShotsOnTarget'] / outfielder_predict['MinutesPlayed']

outfielder_predict = outfielder_predict.drop(['ClubInvolvedPlayed', 'ClubInvolvedWin', 'ClubInvolvedDraw',\
     'ClubInvolvedLoose', 'ClubInvolvedGoalFor', 'ClubInvolvedGoalAgainst', 'ClubInvolvedPoints',\
               'ClubInvolvedAssists', 'ClubInvolvedNonPenaltyGoal', 'ClubInvolvedYellowCard', 'ClubInvolvedRedCard',\
                     'ClubInvolvedNumPlayersUsed',\
                         'ClubInvolvedShotsOnTarget', 'ClubInvolvedGoalPerShotOnTarget', 'ClubInvolvedShotOnTargetAgainst', 'ClubInvolvedCleanSheets',\
                              'MatchesPlayed', 'Starts', 'MatchesPlayed', 'Goals', 'Assists', 'NonPenaltyGoals', 'YellowCards',\
                                   'RedCards', 'ShotsOnTarget',  'GoalsPer90', 'AssistsPer90', 'GoalsAssistsPer90', 'ShotsOnTargetPer90'], axis=1)

outfielder_predict['CorrMarketValue'] = outfielder_predict['NewMarketValue']

result_df = outfielder_predict[['Name', 'ClubInvolved', 'MainPosition',  'ID', 'Nationality']]

outfielder_predict = outfielder_predict.drop([ 'Name', 'ClubInvolved', 'ContractGood', 'Player',\
    'Nation', 'FbrefClub', 'GoalPerShotOnTarget', 'Pos', 'MarketValue', 'Composure', 'ID', 'Joined', 'NewMarketValue', 'Nationality'], axis=1)

outfielder_predict = outfielder_predict[feature_list]

In [180]:
#encode
outfielder_predict_cbe = cbe_encoder.transform(outfielder_predict)

In [181]:
#impute
outfielder_predict_cbe_imp = imputer.transform(outfielder_predict_cbe)
outfielder_predict_cbe_imp = pd.DataFrame(data = outfielder_predict_cbe_imp, columns = outfielder_predict_cbe.columns)

In [182]:
#standardization
outfielder_predict_imp_st = scaler.transform(outfielder_predict_cbe_imp)
outfielder_predict_imp_st = pd.DataFrame(data = outfielder_predict_imp_st, columns = outfielder_predict_cbe_imp.columns)

In [183]:
outfielder_predict_imp_st['DEGREE_IT_SERIE_A_MUL'] = outfielder_predict_imp_st['DEGREE'] * outfielder_predict_imp_st['IT_SERIE_A']

In [185]:
#predict 
model.fit(X_res, y_res)
result_df['confidence'] = model.predict_proba(outfielder_predict_imp_st)[:, 1]
result_df.rename(columns={'ClubInvolved': 'Team', 'MainPosition':'Position', 'confidence':'Probability'}, inplace=True)

#result_df.to_csv(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/TransferPredict/transfer_predict_api/transferPredict/transferPredictApp/Predictions/Data/result_SP_res.csv")


result_df.to_excel(f"/Users/{os.environ['USER']}/Documents/Uni/Szakdoga/Adatok/result_IT_voting.xlsx")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [60]:
result_df.head()

,Name,Team,Position,ID,Nationality,Probability
0,L. Messi,FC Barcelona,Forward,158023,Argentina,0.404927
1,Cristiano Ronaldo,Juventus,Forward,20801,Portugal,0.532061
3,K. De Bruyne,Manchester City,Midfielder,192985,Belgium,0.025140
4,Neymar Jr,Paris Saint-Germain,Forward,190871,Brazil,0.871285
5,R. Lewandowski,FC Bayern München,Forward,188545,Poland,0.562884


In [186]:
pos = len(result_df[(result_df.Name.isin(['Z. Vanheusden',
'D. Dumfries',
'M. Darmian',
'H. Çalhanoğlu',
'F. Tomori',
'Y. Adli',
'S. Tonali',
'F. Ballo-Touré',
'O. Giroud',
'C. Romero',
'T. Koopmeiners',
'M. Lovato',
'D. Zappacosta',
'D. Vlahović',
'D. Zakaria',
'M. Ihattaren',
'M. Politano',
'Juan Jesus',
'E. Hysaj',
'Pedro',
'T. Abraham',
'M. Kumbulla',
'E. Shomurodov',
'Ibañez',
'A. La Gumina',
'A. Candreva',
'A. Conti',
'M. De Luca',
'G. Magnani',
'F. Ceccherini',
'L. Czyborra',
'S. Hefti',
'M. Frendrup',
'A. Touré',
'F. Caicedo',
'C. Ekuban',
'N. Maksimović',
'A. Buksa',
'S. Sabelli',
'K. Bonifazi',
'M. Arnautović',
'L. Samardžić',
'M. Silvestre',
'B. Soppy',
'I. Success',
'Arthur Cabral',
'J. Ikoné',
'Igor',
'J. Antiste',
'K. Amian'
'D. Nikolaou', 'D. Verde', 'R. Marin', 'G. Zappa', 'E. Goldaniga', 'K. Baldé',
'M. Cáceres', 'D. Zima', 'L. Tonelli', 'N. Haas', 'A. Ismajli', 'L. Henderson',
'G. Crociata', 'R. Fiamozzi', 'Simy', 'R. Gagliolo', 'F. Fazio', 'J. Obi',
'L. Coulibaly', 'T. Henry', 'M. Ullmann', 'R. Haps', 'D. Heymans']))&(result_df.Probability>=0.5)])

In [187]:
positives = len(result_df[(result_df.Probability>=0.5)])
recall = round(pos/50, 4)
precision = round(pos/positives, 4)

false_pos = positives - pos
false_neg = 50 - pos
acc = round((11702 - false_pos - false_neg) / 11702,4)

print(f'Recall: {recall}\nPrecision: {precision}\nAccuracy: {acc}')

Recall: 0.72
Precision: 0.0627
Accuracy: 0.9528
